In [1]:
import huggingface_hub
import os
from datasets import load_datasett
import ollama

/Users/edwin/git/Transcendence_LLM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
huggingface_hub.login(os.environ['HF_API_TOKEN'])
def load_eval_dataset(dataset_name="squad_v2"):
    # Load the SQuAD v2 dataset by default
    file_path = "Meta-Llama-3.1-8B-evals/Details_squad_2024-07-22T14-58-08.291117.parquet.gzip"
    dataset = load_dataset(dataset_name, data_files=file_path, split="train")
    return dataset


def generate_llm(prompt: str, model: str = "llama3.1", temperature=0.01) -> str:
    return ollama.generate(
        model=model, prompt=prompt, options=ollama.Options(temperature=temperature, num_predict=32)
    )["response"]

dataset = load_eval_dataset(dataset_name="meta-llama/Meta-Llama-3.1-8B-evals")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/edwin/.cache/huggingface/token
Login successful


In [3]:
dataset

Dataset({
    features: ['task_type', 'task_name', 'subtask_name', 'input_question', 'input_choice_list', 'input_final_prompts', 'input_correct_responses', 'output_prediction_text', 'output_parsed_answer', 'output_choice_completions', 'output_choice_negative_log_likelihoods', 'output_metrics', 'is_correct', 'input_question_hash', 'input_final_prompts_hash', 'benchmark_label', 'eval_config'],
    num_rows: 11873
})

In [4]:
generate_llm("What is the capital of Texas?")


'The capital of Texas is Austin.'

In [5]:
'{hi}'.format(hi='hello')

'hello'

In [6]:
import pickle
from collections import defaultdict
import time
import tqdm
############################################
# Attempt 3:
############################################
predictions = defaultdict(list)
references = defaultdict(list)


eval_prompt = """ 
You are a evaluator that needs to compare if response1 matches any of the other_responses to determine if they are the same or not.

response1: {llm_response}

other_responses: {input_correct_responses}

Answer just "yes" or "no" to the question: "Do the responses match?"
"""

def evaluate_llm(temp=0.01, total_evals=1, model="llama3.1"):
    evaluation_history = []

    for i, example in tqdm.tqdm(enumerate(dataset[:total_evals])):
        generation = generate_llm(example["input_question"], model=model, temperature=temp)
        try:
            generation = generation.split('\n\n')[0]
        except:
            pass
        # print('START GENERATION:\n',  generation)
        # print('END GENERATION\n')
        # print('Correct responses:', example["input_correct_responses"])

        correct = generate_llm(
            eval_prompt.format(
                llm_response=generation,
                input_correct_responses=example["input_correct_responses"],
            ),
            model='llama3.1',
        )
        evaluation_history.append(
            {
                "id": example["input_question_hash"],
                "generation": generation,
                "input_correct_responses": example["input_correct_responses"],
                "correct": correct.lower().strip('.'),
                "temperature": temp,
            }
        )
    # serialize to disk
    with open(f"evaluation_history_{model=}_{temp=}_{total_evals=}.pkl", "wb") as f:
        pickle.dump(evaluation_history, f)
    return evaluation_history


In [7]:

all_evaluations ={}
for model in ["gemma2:2b-text-fp16", "mistral:text", "gemma2:9b-text-fp16", "qwen2:7b-text"]:
    for temp in [0.001, 0.25, 0.5, 0.75, 1.0, 1.5]:
        evaluation_history = evaluate_llm(temp, total_evals=1000, model='mistral:text')
        all_evaluations[f"{temp}_{model}"] = evaluation_history

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
models = ["gemma2:2b-text-fp16", "mistral:text", "gemma2:9b-text-fp16", "qwen2:7b-text"]

In [ ]:
all_evaluations



{0.001: [{'id': '8bda73a1a16177a7b1a27cd99ca72b17ba96f04c86c36a256e2abea3e1b3eeb1',
   'generation': ' John Paul II',
   'input_correct_responses': ['john paul ii', 'john paul ii', 'john paul ii'],
   'correct': 'yes',
   'temperature': 0.001},
  {'id': '07fac974d3c5714d8a990a8b1fdd61838939c6139265307baf0770058dd1f7aa',
   'generation': ' 1851',
   'input_correct_responses': ['1851', 'in 1851', '1851'],
   'correct': 'yes',
   'temperature': 0.001},
  {'id': 'd568ae5d2c9056b505e6ee7c42ccc4d612c2b28d4663720298a38bda20f027e4',
   'generation': ' 20 million ounces',
   'input_correct_responses': ['20 million ounces',
    '20 million ounces',
    '20 million ounces'],
   'correct': 'yes',
   'temperature': 0.001},
  {'id': '9c42a3f7f7debe911ba20717c6ec1ff5176f848eaf17d01c5c82927e196fb31b',
   'generation': ' 1 July 1851',
   'input_correct_responses': ['not in background'],
   'correct': 'no',
   'temperature': 0.001},
  {'id': 'e5e3bae01a94b7d75508c24c2fde048ad295188d160d4dee3ddbd0023e130

In [ ]:
import pandas as pd

df001 = pd.DataFrame(all_evaluations[0.001])
df001

,id,generation,input_correct_responses,correct,temperature
0,8bda73a1a16177a7b1a27cd99ca72b17ba96f04c86c36a...,John Paul II,"[john paul ii, john paul ii, john paul ii]",yes,0.001
1,07fac974d3c5714d8a990a8b1fdd61838939c613926530...,1851,"[1851, in 1851, 1851]",yes,0.001
2,d568ae5d2c9056b505e6ee7c42ccc4d612c2b28d466372...,20 million ounces,"[20 million ounces, 20 million ounces, 20 mill...",yes,0.001
3,9c42a3f7f7debe911ba20717c6ec1ff5176f848eaf17d0...,1 July 1851,[not in background],no,0.001
4,e5e3bae01a94b7d75508c24c2fde048ad295188d160d4d...,Not in background,[not in background],yes,0.001
...,...,...,...,...,...
95,d6f86f97949e50afcd2580a38f39f0743a9773acc731f2...,Not in background,[not in background],yes,0.001
96,93ae398b3aab76904fafff74151b2d1c721dc67c70ddd5...,Not in background,[not in background],yes,0.001
97,04ac4ae3c2a36c7e078a64530a1f9440ee4abee2de20ae...,Members of the Victorian Parliament,[not in background],no,0.001
98,c8059e2fd92eb9e34d064c12e495e764a9635f0657ed25...,Miners,[not in background],no,0.001


In [ ]:
import pandas as pd

df1 = pd.DataFrame(all_evaluations[1.0])
df1

,id,generation,input_correct_responses,correct,temperature
0,8bda73a1a16177a7b1a27cd99ca72b17ba96f04c86c36a...,John Paul II,"[john paul ii, john paul ii, john paul ii]",yes,1.0
1,07fac974d3c5714d8a990a8b1fdd61838939c613926530...,1851,"[1851, in 1851, 1851]",yes,1.0
2,d568ae5d2c9056b505e6ee7c42ccc4d612c2b28d466372...,20 million ounces of gold,"[20 million ounces, 20 million ounces, 20 mill...",yes,1.0
3,9c42a3f7f7debe911ba20717c6ec1ff5176f848eaf17d0...,1 July 1851,[not in background],no,1.0
4,e5e3bae01a94b7d75508c24c2fde048ad295188d160d4d...,Not in background,[not in background],yes,1.0
...,...,...,...,...,...
95,d6f86f97949e50afcd2580a38f39f0743a9773acc731f2...,Not in background,[not in background],yes,1.0
96,93ae398b3aab76904fafff74151b2d1c721dc67c70ddd5...,Not in background,[not in background],yes,1.0
97,04ac4ae3c2a36c7e078a64530a1f9440ee4abee2de20ae...,Members of the Victorian Parliament,[not in background],no,1.0
98,c8059e2fd92eb9e34d064c12e495e764a9635f0657ed25...,Miners,[not in background],no,1.0
